# Commands and State Synchronization in Narupa

This notebook explores the concepts of commands and state synchronization in Narupa. You'll learn

* How to set up custom commands on a Narupa server. 
* How to set shared variables and objects between clients via a Narupa server.
* How to write a toy gravity simulation

## Commands

### What is a command?

In Narupa, a *command* is a function on the server that can be run by a client. These can be used to customize a server,
or create entirely new applications. They are used in lots of places. For example, telling the simulation to pause, play, reset and step uses them! 

Let's create a Narupa server. All servers and applications that derive from a NarupaServer have the ability to run commands.

In [1]:
from narupa.core import NarupaServer

In [2]:
server = NarupaServer(address='localhost', port=0)

### A Basic Command

Let's set up a very simple command

In [3]:
def hello_world():
    print('Hello World!')

To register it, we have to provide a unique name for the command, which will be used by clients

In [5]:
server.register_command('hello', hello_world)

ValueError: Command with name hello has already been registered.

Now let's connect a client to the server.

In [6]:
from narupa.core import NarupaClient

In [8]:
client = NarupaClient.insecure_channel(address=server.address, port=server.port)

We can ask what commands are available.

In [13]:
client.update_available_commands()

{'hello': <narupa.command.command_info.CommandInfo at 0x23477a3a808>}

Let's run it! 

In [16]:
client.run_command('hello')

Hello World!


{}

You'll notice that the output of the command was printed in the cell, and it returned an empty dictionary. This leads us nicely onto more advanced commands

### Taking Arguments and Returning Results

A command is expected to have the following signature

In [18]:
from typing import Dict
def command(**kwargs) -> Dict[str, object]:
    pass

So this means it can define keyword arguments, and can return a dictionary of objects with string keys. For practical reasons of transmitting over the wire, the dictionary must only contain simple objects, such as numbers, booleans, strings and lists and dictionaries of these things. This is because it is internally converted to a protobuf Struct, which is similar to a JSON file.

Let's make a more complicated example

In [38]:
def pythagoras(a=1, b=1):
    c = a**2 + b**2 
    return {'c': c}

In [20]:
server.register_command('pythagoras', pythagoras)

Let's run the command again - we could update the commands on the client, but we know it's registered

In [21]:
client.run_command('pythagoras')

{'r': 2.0}

This time, we receive a dictionary of results, as defined in the method. Let's set some arguments:

In [23]:
client.run_command('pythagoras', a=1, b=2)

{'r': 5.0}

Verifying this result is left as an exercise. What happens if we send incorrect arguments?

In [34]:
import grpc
try:
    client.run_command('pythagoras', c=2)
except grpc.RpcError as e:
    print(e.details())

Exception calling application: pythagoras() got an unexpected keyword argument 'c'


Cool! Even if the client was in a different language, on a different computer, we can see why we messed up!

When registering the command, you can optionally define some defaults. Let's remove our previous definition, and add it with some defaults. This can be useful if you want to set default behaviour for some method that isn't directly under your control

In [42]:
server.unregister_command('pythagoras')
server.register_command('pythagoras', pythagoras, {'a':2,'b':2})

In [43]:
client.run_command('pythagoras')

{'c': 8.0}

It also gives the client some information about what the command accepts:

In [46]:
client.update_available_commands()
client.available_commands['pythagoras'].arguments

{'a': 2.0, 'b': 2.0}

In [184]:
client.close()
server.close()

### Using commands to drive an application

Commands become powerful when they are used to *change* something on the server, making the server interactive. 

The [trajectory viewer](../mdanalysis/mdanalysis_trajectory.ipynb) example demonstrates this. 

## Synchronising State

Narupa uses a client-server model, which means clients do not talk to eachother directly. Never-the-less, it is often important to synchronise information across multiple clients, for example the position of the simulation, or the current state of one's avatar (e.g. if you're in a menu).

We achieve this with a shared state dictionary that clients can update.

In [429]:
server = NarupaServer(address='localhost', port=0)

In [430]:
client = NarupaClient.insecure_channel(address='localhost', port=server.port)

The following command makes the client listen to any changes on the shared state dictionary.

In [431]:
client.subscribe_all_state_updates()

At first, this dictionary is empty:

In [447]:
client.copy_state()

{'a': 2.0}

Let's connect a second client

In [448]:
second_client = NarupaClient.insecure_channel(address='localhost', port=server.port)

In [449]:
second_client.subscribe_all_state_updates()

Now, the first client will make a change to the dictionary, and we'll see both clients will get the update.

In [450]:
from narupa.state.state_dictionary import DictionaryChange

In [451]:
changes = DictionaryChange({'a':2}, [])

In [452]:
client.attempt_update_state(changes)

True

In [453]:
client.copy_state()

{'a': 2.0}

In [454]:
second_client.copy_state()

{'a': 2.0}

The DictionaryChange object took a second argument - removals. We can use that to remove things from the shared state.

In [455]:
changes = DictionaryChange({}, ['a'])
second_client.attempt_update_state(changes)

True

Now, the dictionary is empty. 

In [456]:
client.copy_state()

{}

In [457]:
second_client.copy_state()

{}

What if both clients try to update at the same time, or what if I don't want someone else to mess with something?
We handle this by locking the key while changes are made, so only one client can edit fields at a time:

In [462]:
# We aquire a lock on the key 'a' for 10 seconds.
got_lock = client.attempt_update_locks({'a': 10})
print(f'Client has a lock: {got_lock}')
successfully_updated = second_client.attempt_update_state(changes)
print(f'Second client updated state: {successfully_updated}')

Client has a lock: True
Second client updated state: False


The dictionary accepts anything that can be represented as a protobuf Struct, so you can set up complicated things:

In [468]:
changes = DictionaryChange({
    'party':{'pokemon':['charmander','squirtle','bulbasaur','pikachu']},
    'battle':{
        'name':'charmander',
        'type':'fire',
        'level': 7,
        'abilities':['scratch', 'growl']
    }
}, [])
second_client.attempt_update_state(changes)

True

In [473]:
import pprint # pretty print!
pprint.pprint(client.copy_state())

{'battle': {'abilities': ['scratch', 'growl'],
            'level': 7.0,
            'name': 'charmander',
            'type': 'fire'},
 'party': {'pokemon': ['charmander', 'squirtle', 'bulbasaur', 'pikachu']}}


# Next Steps

* See practical examples of commands in the [trajectory viewer](../mdanalysis/trajectory_viewer.ipynb)
* Learn more about how [servers](servers.ipynb) are constructed. 